In [ ]:
# https://github.com/ghislainv/forestatrisk/tree/master/forestatrisk/data/compute


## Libraries

In [ ]:
import os

os.environ["PROJ_LIB"] = "/usr/share/proj"
os.environ["GDAL_DATA"] = "/usr/share/gdal"


In [ ]:
# GDAL optimizations
import multiprocessing
import os

cpu_count: int = multiprocessing.cpu_count()
num_cores: int = cpu_count - 2
os.environ["GDAL_NUM_THREADS"] = f"{num_cores}"
os.environ["GDAL_CACHEMAX"] = "1024"


In [ ]:
import psutil

memory_gb = psutil.virtual_memory().total / (1024**3)
print(psutil.virtual_memory().total)


In [ ]:
import pathlib
import ee
import fiona
import geemap
import geopandas as gpd
import pandas as pd
from fiona.transform import transform_geom
from osgeo import gdal
from pyproj import Proj, transform


## Connect folders

In [ ]:
root_folder: Path = Path.cwd().parent
downloads_folder: Path = root_folder / "data"
downloads_folder.mkdir(parents=True, exist_ok=True)


## Set user parameters

In [ ]:
# project_name = "MTQ"
# user_defined_epsg_code = "5490"
# years = [2005, 2010, 2015]
# tree_cover_threshold = 10
# forest_source = "gfc"  ##gfc, tmf, mapbiomas
# random_seed = 1


In [ ]:
project_folder = downloads_folder / project_name
project_folder.mkdir(parents=True, exist_ok=True)
data_raw_folder = project_folder / "data_raw"
data_raw_folder.mkdir(parents=True, exist_ok=True)
processed_data_folder = project_folder / "data"
processed_data_folder.mkdir(parents=True, exist_ok=True)


## Calculate epsg code

In [ ]:
import fiona
from shapely.geometry import shape


def get_centroid(shapefile_path):
    """
    Get the centroid of the first feature in a shapefile using Shapely and Fiona.

    Parameters:
        shapefile_path (str): Path to the input shapefile.

    Returns:
        tuple: A tuple containing the latitude and longitude of the centroid.
    """
    # Open the shapefile
    with fiona.open(shapefile_path, "r") as shapefile:
        # Get the first feature
        first_feature = next(iter(shapefile))

        # Convert the feature geometry to a Shapely geometry object
        geom = shape(first_feature["geometry"])

        # Calculate the centroid
        centroid = geom.centroid

        # Get the coordinates of the centroid
        longitude, latitude = centroid.x, centroid.y

    return (longitude, latitude)


In [ ]:
def get_utm_proj_str_from_lat_lon(lon, lat):
    """
    Given a longitude, latitude in WGS84, return the EPSG code as a string
    for the corresponding UTM or UPS projection.

    - UTM: EPSG:326xx (Northern) or EPSG:327xx (Southern)
    - UPS: EPSG:5041 (North, >84°N), EPSG:5042 (South, <–80°S)

    Handles special cases for Norway and Svalbard.
    """
    # UPS zones for polar regions
    if lat >= 84:
        return "EPSG:5041"  # UPS North
    elif lat <= -80:
        return "EPSG:5042"  # UPS South

    # Special cases for Norway and Svalbard
    if lat > 55 and lat < 64 and lon > 2 and lon < 6:
        zone_number = 32
    elif lat > 71 and lon >= 6 and lon < 9:
        zone_number = 31
    elif lat > 71 and ((lon >= 9 and lon < 12) or (lon >= 18 and lon < 21)):
        zone_number = 33
    elif lat > 71 and ((lon >= 21 and lon < 24) or (lon >= 30 and lon < 33)):
        zone_number = 35
    else:
        zone_number = int((lon + 180) / 6) + 1

    if lat >= 0:
        epsg_code = 32600 + zone_number  # Northern Hemisphere
    else:
        epsg_code = 32700 + zone_number  # Southern Hemisphere

    return epsg_code


In [ ]:
aoi_centroid = get_centroid(str(data_raw_folder) + "/" + project_name + "_aoi.shp")
calculated_epsg = get_utm_proj_str_from_lat_lon(aoi_centroid[0], aoi_centroid[1])
calculated_epsg


In [ ]:
epsg_code = user_defined_epsg_code


## Filter files helpers

In [ ]:
def list_files_by_extension(folder_path, file_extensions):
    """
    List all files with specified extensions in the given folder.

    Parameters:
    folder_path (str): The path to the folder where you want to search for files.
    file_extensions (list of str): A list of file extensions to search for (e.g., ['.shp', '.tif']).

    Returns:
    list: A list of file paths with the specified extensions.
    """
    matching_files = []
    try:
        # Check if the provided path is a directory
        if os.path.isdir(folder_path):
            # Iterate over all files in the directory
            for filename in os.listdir(folder_path):
                # Construct full file path
                file_path = os.path.join(folder_path, filename)
                # Check if the file has any of the specified extensions
                if any(filename.lower().endswith(ext) for ext in file_extensions):
                    matching_files.append(file_path)
        else:
            print(f"The provided path '{folder_path}' is not a directory.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return matching_files


In [ ]:
def filter_files(input_files, filter_words, exclude_words=None):
    """
    Filters a list of files based on include and exclude words.

    Parameters:
        input_files (list): List of file paths to be filtered.
        filter_words (list): Words that must be present in the filenames for inclusion.
        exclude_words (list, optional): Words that must not be present in the filenames for exclusion. Defaults to None.

    Returns:
        list: Filtered list of files.
    """
    # Ensure all words are lowercase for case-insensitive comparison
    filter_words = [word.lower() for word in filter_words]
    exclude_words = [word.lower() for word in (exclude_words or [])]

    filtered_files = [
        file
        for file in input_files
        if all(word in os.path.basename(file).lower() for word in filter_words)
        and not any(
            exclude_word in os.path.basename(file).lower()
            for exclude_word in exclude_words
        )
    ]

    return filtered_files


## Reproject Vector Data

In [ ]:
def reproject_shapefile(input_shapefile, output_shapefile, target_epsg):
    # Open the input shapefile
    with fiona.open(input_shapefile, "r") as src:
        # Get the source CRS
        src_crs = src.crs
        # Define the target CRS
        target_crs = f"EPSG:{target_epsg}"

        # Define the transformation function
        project = lambda geom: transform_geom(src_crs, target_crs, geom)

        # Open the output shapefile for writing
        with fiona.open(
            output_shapefile,
            "w",
            crs=target_crs,
            driver="ESRI Shapefile",
            schema=src.schema,
        ) as dst:
            # Iterate over features in the source shapefile
            for feature in src:
                # Transform the geometry and create a new feature
                new_feature = {**feature, "geometry": project(feature["geometry"])}
                # Write the transformed feature to the output shapefile
                dst.write(new_feature)


In [ ]:
def reproject_shp_files(shp_folder, tif_folder, target_epsg):
    """
    Process .shp files by generating corresponding .tif filenames and calling rasterize_vectors.

    Parameters:
    shp_folder (str): The path to the folder containing .shp files.
    tif_folder (str): The path to the folder where .tif files will be saved.
    target_epsg (int): The EPSG code of the target coordinate reference system.
    """
    shp_files = list_files_by_extension(shp_folder, [".shp"])
    for shp_file in shp_files:
        # Extract the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(shp_file))[0]
        # Create the new .tif filename
        reprojected_filename = f"{base_name}_reprojected.shp"
        reprojected_path = os.path.join(tif_folder, reprojected_filename)
        # Call rasterize_vectors with the original and new filenames
        reproject_shapefile(shp_file, reprojected_path, target_epsg)


In [ ]:
reproject_shp_files(data_raw_folder, processed_data_folder, epsg_code)


## Reproject Raster Data

In [ ]:
from osgeo import gdal


def reproject_raster_gdal(input_file, output_file, target_epsg):
    """
    Reprojects a raster file to a specified EPSG code using GDAL and saves it with DEFLATE compression.

    Parameters:
    input_file (str): The path to the input raster file.
    output_file (str): The path where the reprojected raster file will be saved.
    target_epsg (int): The EPSG code of the target coordinate reference system.

    Returns:
    None
    """
    # Open the input dataset
    dataset = gdal.Open(input_file)
    if not dataset:
        raise FileNotFoundError(f"Input file {input_file} not found.")

    # Get projection and geotransform from the original raster
    src_proj = dataset.GetProjection()
    src_geotrans = dataset.GetGeoTransform()
    # print(dataset.GetRasterBand(1).DataType)
    # Create the output dataset with the target EPSG and DEFLATE compression
    driver = gdal.GetDriverByName("GTiff")
    out_dataset = driver.Create(
        output_file,
        dataset.RasterXSize,
        dataset.RasterYSize,
        dataset.RasterCount,
        dataset.GetRasterBand(1).DataType,
        ["COMPRESS=DEFLATE"],
    )

    # Set projection and geotransform for the output dataset
    out_proj = f"EPSG:{target_epsg}"
    out_geotrans = src_geotrans  # Assuming same resolution and extent

    out_dataset.SetProjection(out_proj)
    out_dataset.SetGeoTransform(out_geotrans)

    # Perform reprojection
    gdal.ReprojectImage(
        dataset, out_dataset, src_proj, out_proj, gdal.GRA_NearestNeighbour
    )

    # Close datasets
    dataset = None
    out_dataset = None


In [ ]:
from osgeo import gdal
import multiprocessing


def reproject_raster_gdal_warp(
    input_file: str,
    output_file: str,
    target_epsg: str,
    target_x: int | float = 30,
    target_y: int | float = 30,
    resampling_method: str = "near",
) -> None:
    """
    Reprojects a raster file to a specified EPSG code using GDAL and saves it with DEFLATE compression.

    Parameters:
    input_file (str): The path to the input raster file.
    output_file (str): The path where the reprojected raster file will be saved.
    target_epsg (int): The EPSG code of the target coordinate reference system.

    Returns:
    None
    """

    # Get system information for optimization
    cpu_count = multiprocessing.cpu_count()
    # Determine optimal number of cores (use 4 or less, or available cores)
    num_cores = cpu_count - 2

    # Open the input dataset
    dataset = gdal.Open(input_file)
    if not dataset:
        raise FileNotFoundError(f"Input file {input_file} not found.")

    # Get projection and geotransform from the original raster
    src_proj = dataset.GetProjection()

    # Callback
    param = gdal.WarpOptions(
        warpOptions=["overwrite"],
        srcSRS=src_proj,
        dstSRS=f"EPSG:{target_epsg}",
        targetAlignedPixels=True,
        resampleAlg=resampling_method,
        xRes=target_x,
        yRes=target_y,
        multithread=True,
        warpMemoryLimit=500 * num_cores,
        creationOptions=[
            "COMPRESS=DEFLATE",
            "PREDICTOR=2",
            "BIGTIFF=YES",
            f"NUM_THREADS={num_cores}",
        ],
    )

    # Perform reprojection
    gdal.Warp(output_file, input_file, format="GTiff", options=param)

    # Close datasets
    dataset = None
    out_dataset = None


In [ ]:
import rasterio
from rasterio.warp import Resampling, calculate_default_transform, reproject


def reproject_raster_rasterio(input_file, output_file, target_epsg):
    """
    Reprojects a raster file to a specified EPSG code using Rasterio and saves it with DEFLATE compression.

    Parameters:
    input_file (str): The path to the input raster file.
    output_file (str): The path where the reprojected raster file will be saved.
    target_epsg (int): The EPSG code of the target coordinate reference system.

    Returns:
    None
    """
    with rasterio.open(input_file) as src:
        transform, width, height = calculate_default_transform(
            src.crs, f"EPSG:{target_epsg}", src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update(
            {
                "crs": f"EPSG:{target_epsg}",
                "transform": transform,
                "width": width,
                "height": height,
                "compress": "deflate",  # Enable DEFLATE compression
            }
        )

        with rasterio.open(output_file, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=f"EPSG:{target_epsg}",
                    resampling=Resampling.nearest,
                )


In [ ]:
def reproject_tiff_files_near(input_folder, tif_folder, target_epsg):
    """
    Reproject .tif files based on data type

    Parameters:
    input_folder (str): The path to the folder containing .shp files.
    tif_folder (str): The path to the folder where .tif files will be saved.
    target_epsg (int): The EPSG code of the target coordinate reference system.

    """
    raster_files = list_files_by_extension(input_folder, [".tiff", ".tif"])
    # Define the words to filter by
    filter_words = ["town", "rivers", "roads", "wdpa", forest_source]

    # Filter the raster files based on the presence of any of the filter words in their filenames
    filtered_raster_files = [
        file
        for file in raster_files
        if any(word in os.path.basename(file).lower() for word in filter_words)
    ]
    for raster_file in filtered_raster_files:
        # Extract the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(raster_file))[0]
        # Create the new .tif filename
        tif_filename = f"{base_name}_reprojected.tif"
        tif_path = os.path.join(tif_folder, tif_filename)
        # Call rasterize_vectors with the original and new filenames
        reproject_raster_gdal_warp(raster_file, tif_path, target_epsg)


In [ ]:
def reproject_tiff_files_bilinear(input_folder, tif_folder, target_epsg):
    """
    Reproject .tif files based on data type.

    Parameters:
    input_folder (str): The path to the folder containing .shp files.
    tif_folder (str): The path to the folder where .tif files will be saved.
    target_epsg (int): The EPSG code of the target coordinate reference system.

    """
    raster_files = list_files_by_extension(input_folder, [".tiff", ".tif"])
    # Define the words to filter by
    filter_words = ["altitude", "slope"]

    # Filter the raster files based on the presence of any of the filter words in their filenames
    filtered_raster_files = [
        file
        for file in raster_files
        if any(word in os.path.basename(file).lower() for word in filter_words)
    ]
    for raster_file in filtered_raster_files:
        # Extract the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(raster_file))[0]
        # Create the new .tif filename
        tif_filename = f"{base_name}_reprojected.tif"
        tif_path = os.path.join(tif_folder, tif_filename)
        # Call rasterize_vectors with the original and new filenames
        reproject_raster_gdal_warp(
            raster_file, tif_path, target_epsg, resampling_method="bilinear"
        )


In [ ]:
reproject_tiff_files_bilinear(data_raw_folder, processed_data_folder, epsg_code)


In [ ]:
reproject_tiff_files_near(data_raw_folder, processed_data_folder, epsg_code)


## Rasterize Vector Data

In [ ]:
import fiona
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds


def rasterize_binary_orig(vector_path, output_raster_path, pixel_size=30):
    """
    Rasterize a vector file to a raster file using Rasterio.

    Parameters:
        vector_path (str): Path to the input vector file.
        output_raster_path (str): Path to the output raster file.
        pixel_size (float): Pixel size of the output raster. Default is 30.
    """
    # Open the vector file
    with fiona.open(vector_path, "r") as shapefile:
        # Get the bounding box of the vector file
        bounds = shapefile.bounds

        # Ensure valid bounding box dimensions
        if (bounds[2] - bounds[0]) == 0 or (bounds[3] - bounds[1]) == 0:
            raise ValueError(
                "Invalid bounding box dimensions. The bounding box has zero width or height."
            )

        # Calculate the width and height of the output raster
        width = int((bounds[2] - bounds[0]) / pixel_size)
        height = int((bounds[3] - bounds[1]) / pixel_size)

        # Create an empty array to hold the raster data
        raster_data = np.zeros((height, width), dtype=np.uint8)

        # Rasterize each feature in the shapefile
        for feature in shapefile:
            geom = feature["geometry"]
            value = 1  # Set all features to 1
            shapes = [(geom, value)]
            rasterize(
                shapes=shapes,
                out=raster_data,
                fill=0,
                transform=from_bounds(*bounds, width, height),
            )

    # Write the raster data to a file
    with rasterio.open(
        output_raster_path,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype=np.uint8,
        crs=shapefile.crs,
        transform=from_bounds(*bounds, width, height),
        compress="deflate",
    ) as dst:
        dst.write(raster_data, 1)


In [ ]:
import fiona
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds


def rasterize_binary(vector_path, extent_file, output_raster_path, pixel_size=30):
    """
    Rasterize a vector file to a raster file using Rasterio.
    Parameters:
        vector_path (str): Path to the input vector file.
        output_raster_path (str): Path to the output raster file.
        fcc_file (str): Path to the reference raster file for resolution and extent.
        pixel_size (float): Pixel size of the output raster. Default is 30.
    """
    # Open the reference raster file
    with rasterio.open(extent_file) as src:
        gt = src.transform
        xmin, ymin, xmax, ymax = src.bounds
        xres = gt[0]
        yres = -gt[4]

    # Calculate the width and height of the output raster based on the reference raster's resolution
    width = int((xmax - xmin) / xres)
    height = int((ymax - ymin) / yres)

    # Create an empty array to hold the raster data
    raster_data = np.zeros((height, width), dtype=np.uint8)

    # Open the vector file
    with fiona.open(vector_path, "r") as shapefile:
        # Rasterize each feature in the shapefile
        for feature in shapefile:
            geom = feature["geometry"]
            value = 1  # Set all features to 1
            shapes = [(geom, value)]
            rasterize(
                shapes=shapes,
                out=raster_data,
                fill=0,
                transform=from_bounds(xmin, ymin, xmax, ymax, width, height),
            )

    # Write the raster data to a file
    with rasterio.open(
        output_raster_path,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype=np.uint8,
        crs=shapefile.crs,
        transform=from_bounds(xmin, ymin, xmax, ymax, width, height),
        compress="deflate",
    ) as dst:
        dst.write(raster_data, 1)


In [ ]:
import fiona
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds


def rasterize_unique_id(input_file, extent_file, output_file):
    """Rasterizing subjurisdictions using Rasterio.
    Parameters:
        input_file: Input shapefile with subjurisdictions.
        fcc_file: Input reference file for resolution and extent.
        output_file: Output raster file with integer id for subjurisdictions.
    """
    # Open the reference raster file
    with rasterio.open(extent_file) as src:
        gt = src.transform
        xmin, ymin, xmax, ymax = src.bounds
        xres = gt[0]
        yres = -gt[4]

    # Read the shapefile
    with fiona.open(input_file, "r") as shp:
        features = [feature["geometry"] for feature in shp]

    # Create an output raster
    transform = from_bounds(xmin, ymin, xmax, ymax, src.width, src.height)
    out_meta = {
        "driver": "GTiff",
        "count": 1,
        "height": src.height,
        "width": src.width,
        "transform": transform,
        "crs": src.crs,
        "dtype": rasterio.uint8,
        "compress": "deflate",
    }

    # Rasterize the features
    out_image = rasterize(
        [(geom, value) for geom, value in zip(features, range(1, len(features) + 1))],
        out_shape=(src.height, src.width),
        fill=0,
        transform=transform,
        dtype=rasterio.uint8,
    )

    # Write the output raster file
    with rasterio.open(output_file, "w", **out_meta) as dst:
        dst.write(out_image, 1)


In [ ]:
def rasterize_shp_files(shp_folder, tif_folder):
    """
    Process .shp files by generating corresponding .tif filenames and calling rasterize_vectors.

    Parameters:
    shp_folder (str): The path to the folder containing .shp files.
    tif_folder (str): The path to the folder where .tif files will be saved.
    target_epsg (int): The EPSG code of the target coordinate reference system.
    """
    shp_files = list_files_by_extension(shp_folder, [".shp"])

    shp_files_binary = filter_files(shp_files, ["aoi"])

    reference_forest_file = filter_files(
        list_files_by_extension(processed_data_folder, [".tiff", ".tif"]),
        ["reprojected", forest_source],
    )[0]

    for shp_file in shp_files_binary:
        # Extract the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(shp_file))[0]
        # Create the new .tif filename
        tif_filename = f"{base_name}.tif"
        tif_path = os.path.join(tif_folder, tif_filename)
        # Call rasterize_vectors with the original and new filenames
        rasterize_binary(shp_file, reference_forest_file, tif_path)

    shp_files_unique = filter_files(shp_files, ["subj"])

    for shp_file in shp_files_unique:
        # Extract the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(shp_file))[0]
        # Create the new .tif filename
        tif_filename = f"{base_name}.tif"
        tif_path = os.path.join(tif_folder, tif_filename)
        # Call rasterize_vectors with the original and new filenames
        rasterize_unique_id(shp_file, reference_forest_file, tif_path)


In [ ]:
rasterize_shp_files(processed_data_folder, processed_data_folder)


## Calculate distance

In [ ]:
import numpy as np
import rasterio
from scipy.ndimage import distance_transform_edt


def distance_to_edge_rasterio(input_file, dist_file):
    """Computing the shortest distance to pixels with given values in a raster file.

    This function computes the shortest distance to pixels with given values in a raster file. Distances generated are in georeferenced coordinates.

    :param input_file: Input raster file.

    :param dist_file: Path to the distance raster file that is created.

    :return: None. A distance raster file is created (see ``dist_file``). Raster data type is UInt32 ([0, 4294967295]). NoData is set to zero.

    """

    # Read input file
    with rasterio.open(input_file) as src:
        src_data = src.read(1)
        transform = src.transform
        crs = src.crs

        # Create a mask array where the values match those specified
        values = 0
        mask = np.isin(src_data, values)

        # Initialize distance array with NoData value (0)
        dist_data = np.zeros_like(src_data, dtype=np.uint32)

        # Calculate the shortest distance (this is a simplified approach)
        dist_data = distance_transform_edt(~mask) * transform.a  # Scale by pixel size

    # Write output file
    with rasterio.open(
        dist_file,
        "w",
        driver="GTiff",
        height=src_data.shape[0],
        width=src_data.shape[1],
        count=1,
        dtype=np.uint32,
        crs=crs,
        transform=transform,
        compress="lzw",
        predictor=2,
    ) as dst:
        dst.write(dist_data, 1)
        dst.nodata = 0


In [ ]:
def distance_to_no_forest_rasterio(input_file, dist_file):
    """
    For each forest pixel (value = 1), compute shortest distance to nearest non-forest pixel (value = 0).
    Preserves original NoData mask: output is 0 where input was NoData.

    :param input_file: Input raster path (forest=1, no forest=0)
    :param dist_file: Output distance raster path
    """
    with rasterio.open(input_file) as src:
        data = src.read(1).astype(np.float32)
        transform = src.transform
        crs = src.crs
        nodata = src.nodata

        # Step 1: Define valid pixels (not NoData)
        if nodata is not None:
            valid_mask = ~np.isclose(data, nodata)
        else:
            valid_mask = np.ones_like(data, dtype=bool)

        # print(f"Valid pixels: {valid_mask.sum()} out of {data.size}")

        # Step 2: Define target pixels (non-forest): value == 0 AND not NoData
        targets = np.logical_and(data == 0, valid_mask)
        print(f"Target (no forest) pixels: {targets.sum()}")

        if not targets.any():
            raise ValueError("No non-forest (value=0) pixels found in valid area!")

        # Step 3: Prepare distance transform input
        # We compute distances from all valid pixels to nearest target (non-forest)
        # So we create a grid where:
        #   - All valid pixels are allowed (True)
        #   - But targets themselves are marked as source → set to False in the EDT input

        edt_input = np.zeros_like(data, dtype=bool)
        edt_input[valid_mask] = True  # Only compute within valid area
        edt_input[targets] = False  # Target pixels: set to False (sources)

        print(f"EDT input size: {edt_input.sum()} pixels")

        if not edt_input.any():
            raise ValueError("No non-target valid pixels left!")

        # Step 4: Compute distance transform in georeferenced units
        try:
            dist_array = distance_transform_edt(edt_input) * abs(transform.a)
        except Exception as e:
            raise RuntimeError(f"Distance transform failed: {e}")

        # Step 5: Output only for valid forest pixels (value == 1), set others to 0
        dist_data = np.zeros_like(data, dtype=np.uint32)

        # Only assign distance where data == 1 AND valid
        forest_pixels = np.logical_and(data == 1, valid_mask)
        # print(f"Forest pixels: {forest_pixels.sum()}")

        if forest_pixels.any():
            dist_data[forest_pixels] = dist_array[forest_pixels].astype(np.uint32)

        # Preserve original NoData in output (set to 0)
        if nodata is not None:
            dist_data[np.isclose(data, nodata)] = 0

    # Write output
    with rasterio.open(
        dist_file,
        "w",
        driver="GTiff",
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=np.uint32,
        crs=crs,
        transform=transform,
        compress="lzw",
        predictor=2,
        nodata=0,
    ) as dst:
        dst.write(dist_data, 1)


In [ ]:
def distance_to_edge_gdal(input_file, dist_file):
    """Computing the shortest distance to pixels with given values in
    a raster file.

    This function computes the shortest distance to pixels with given
    values in a raster file. Distances generated are in georeferenced
    coordinates.

    :param input_file: Input raster file.

    :param dist_file: Path to the distance raster file that is
        created.

    :return: None. A distance raster file is created (see
        ``dist_file``). Raster data type is UInt32 ([0,
        4294967295]). NoData is set to zero.

    """

    # Read input file
    src_ds = gdal.Open(input_file)
    srcband = src_ds.GetRasterBand(1)

    # Create raster of distance
    drv = gdal.GetDriverByName("GTiff")
    dst_ds = drv.Create(
        dist_file,
        src_ds.RasterXSize,
        src_ds.RasterYSize,
        1,
        gdal.GDT_UInt32,
        ["COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES"],
    )
    dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
    dst_ds.SetProjection(src_ds.GetProjectionRef())
    dstband = dst_ds.GetRasterBand(1)

    # Compute distance
    values = [0]
    val_as_string = ",".join([str(i) for i in values])
    val = "VALUES=" + val_as_string
    gdal.ComputeProximity(srcband, dstband, [val, "DISTUNITS=GEO"])

    # Set nodata value
    dstband.SetNoDataValue(0)

    # Delete objects
    srcband = None
    dstband = None
    del src_ds, dst_ds


In [ ]:
import numpy as np
from osgeo import gdal


def distance_to_edge_gdal_revised(
    input_file, dist_file, values=0, nodata=4294967295, input_nodata=True, verbose=False
):
    """Computes the shortest distance to given pixel values in a raster,
    while preserving the original nodata mask in the output."""

    # Read input file
    src_ds = gdal.Open(input_file)
    srcband = src_ds.GetRasterBand(1)

    # Create raster of distance
    drv = gdal.GetDriverByName("GTiff")
    dst_ds = drv.Create(
        dist_file,
        src_ds.RasterXSize,
        src_ds.RasterYSize,
        1,
        gdal.GDT_UInt32,
        ["COMPRESS=DEFLATE", "PREDICTOR=2", "BIGTIFF=YES"],
    )
    dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
    dst_ds.SetProjection(src_ds.GetProjection())
    dstband = dst_ds.GetRasterBand(1)

    # Use_input_nodata
    ui_nodata = "YES" if input_nodata else "NO"

    # Compute distance
    val = "VALUES=" + str(values)
    use_input_nodata = "USE_INPUT_NODATA=" + ui_nodata
    cb = gdal.TermProgress_nocb if verbose else 0
    gdal.ComputeProximity(
        srcband, dstband, [val, use_input_nodata, "DISTUNITS=GEO"], callback=cb
    )

    # Inlcude values as part of the mask
    # Read data as array
    src_arr = srcband.ReadAsArray()
    dst_arr = dstband.ReadAsArray()
    # Mask nodata
    # Ensure values is iterable (list/ndarray)
    values = [values] if np.isscalar(values) else values
    dst_arr[np.isin(src_arr, values)] = nodata
    # Write back masked result
    dstband.WriteArray(dst_arr)

    # Set nodata value
    dstband.SetNoDataValue(nodata)

    # Flush to disk
    dstband.FlushCache()
    dst_ds.FlushCache()

    # Clean up
    srcband = None
    dstband = None
    del src_ds, dst_ds


In [ ]:
import numpy as np
import rasterio
from scipy.ndimage import distance_transform_edt


def distance_from_outside(input_file, output_file):
    # Open the input raster file
    with rasterio.open(input_file) as src:
        roads = src.read(1)
        mask = src.read_masks(1) == 0  # True where pixels are masked (nodata)

        # Identify road and non-road pixels
        no_roads = roads == 0

        # Calculate distance from each no-road pixel to the nearest road pixel
        distances = distance_transform_edt(no_roads, sampling=src.transform.a)

        # Apply the original mask to the distance array
        distances = np.where(
            mask, src.nodata if src.nodata is not None else np.nan, distances
        )

    # Write the output raster file
    with rasterio.open(
        output_file,
        "w",
        driver="GTiff",
        height=distances.shape[0],
        width=distances.shape[1],
        count=1,
        dtype=rasterio.float32,
        crs=src.crs,
        transform=src.transform,
        nodata=src.nodata if src.nodata is not None else None,
    ) as dst:
        dst.write(distances.astype(rasterio.float32), 1)


In [ ]:
def distance_from_outside_gdal(input_file, dist_file):
    """
    Compute shortest distance from each non-road pixel (value=0) to nearest road pixel (value=1),
    respecting NoData values and geospatial metadata.

    This function now matches `distance_from_outside` in behavior:
      - Only outputs distances where input == 0 AND valid
      - Preserves original NoData mask
      - Uses georeferenced units via DISTUNITS=GEO

    :param input_file: Input raster (roads=1, no roads=0)
    :param dist_file: Output distance raster path (GTiff)
    """
    from osgeo import gdal

    # Open input dataset
    src_ds = gdal.Open(input_file, gdal.GA_ReadOnly)
    if not src_ds:
        raise FileNotFoundError(f"Could not open {input_file}")

    srcband = src_ds.GetRasterBand(1)

    x_size = src_ds.RasterXSize
    y_size = src_ds.RasterYSize
    geotransform = src_ds.GetGeoTransform()
    projection = src_ds.GetProjectionRef()
    nodata_value = srcband.GetNoDataValue()

    # Read data as numpy array
    data_array = srcband.ReadAsArray().astype(np.float32)

    # Create valid mask: not NoData and value == 0 (source)
    if nodata_value is not None:
        valid_mask = (data_array != nodata_value) & (data_array == 0)
    else:
        valid_mask = data_array == 0

    # Step 1: Compute distance from value=0 to nearest value=1
    # We need: source = value=0 → so use VALUES=0 in ComputeProximity

    # Create temporary output dataset for proximity result
    drv = gdal.GetDriverByName("GTiff")
    dst_ds = drv.Create(
        dist_file,
        x_size,
        y_size,
        1,
        gdal.GDT_UInt32,
        ["COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES"],
    )
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(projection)
    dstband = dst_ds.GetRasterBand(1)

    # Set NoData value to 0 (we'll overwrite it later per input mask)
    dstband.SetNoDataValue(0)

    # Compute proximity from pixels with value=0
    values = [0]
    val_as_string = ",".join([str(i) for i in values])
    gdal.ComputeProximity(
        srcband, dstband, ["VALUES=" + val_as_string, "DISTUNITS=GEO"]
    )

    # Read computed distances
    dist_array = dstband.ReadAsArray().astype(np.uint32)

    # Step 2: Final masking — only keep output where input was value=0 AND valid
    final_data = np.zeros_like(dist_array, dtype=np.uint32)

    if valid_mask.any():
        final_data[valid_mask] = dist_array[valid_mask]

    # Step 3: Apply original NoData mask
    if nodata_value is not None:
        no_data_mask = data_array == nodata_value
        final_data[no_data_mask] = 0  # or set to nodata later

    # Write output with correct data type and NoData setting
    dstband.WriteArray(final_data)

    # Now, write the dataset with proper NoData value in metadata
    # But since we're using UInt32, and want to preserve original NoData,
    # we must set it explicitly. However, GDAL doesn't allow NaN.
    # So use 0 as NoData only if needed.

    # To match rasterio output (which supports float32 with actual nodata), we should
    # write the final_data as Float32 instead of UInt32 to support `nodata` correctly

    # 🔥 IMPORTANT: Change output type to Float32 to allow proper NoData handling
    dst_ds = None  # Close previous dataset
    dst_ds = drv.Create(
        dist_file,
        x_size,
        y_size,
        1,
        gdal.GDT_Float32,
        ["COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES"],
    )
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(projection)

    out_band = dst_ds.GetRasterBand(1)
    # Convert final_data to float32
    final_float = final_data.astype(np.float32)
    out_band.WriteArray(final_float)

    if nodata_value is not None:
        out_band.SetNoDataValue(nodata_value)
    else:
        out_band.SetNoDataValue(None)  # optional

    # Clean up
    srcband = None
    out_band = None
    del src_ds, dst_ds

    print(f"Distance raster saved to: {dist_file}")


In [ ]:
def distance_to_edge_gdal_no_mask(
    input_file,
    dist_file,
    values=0,
    nodata=0,
    max_distance_value=4294967295,
    input_nodata=True,
    verbose=False,
):
    """Computes the shortest distance to given pixel values in a raster,
    while preserving the original nodata mask in the output."""

    # Read input file
    src_ds = gdal.Open(input_file)
    srcband = src_ds.GetRasterBand(1)

    # Create raster of distance
    drv = gdal.GetDriverByName("GTiff")
    dst_ds = drv.Create(
        dist_file,
        src_ds.RasterXSize,
        src_ds.RasterYSize,
        1,
        gdal.GDT_UInt32,
        ["COMPRESS=DEFLATE", "PREDICTOR=2", "BIGTIFF=YES"],
    )
    dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
    dst_ds.SetProjection(src_ds.GetProjection())
    dstband = dst_ds.GetRasterBand(1)

    # Use_input_nodata
    ui_nodata = "YES" if input_nodata else "NO"

    # Compute distance
    val = "VALUES=" + str(values)
    use_input_nodata = "USE_INPUT_NODATA=" + ui_nodata
    max_distance = "MAXDIST=" + str(max_distance_value)
    distance_nodata = "NODATA=" + str(nodata)
    cb = gdal.TermProgress_nocb if verbose else 0
    gdal.ComputeProximity(
        srcband,
        dstband,
        [val, use_input_nodata, max_distance, distance_nodata, "DISTUNITS=GEO"],
        callback=cb,
    )

    # Set nodata value
    dstband.SetNoDataValue(max_distance_value)

    # Flush to disk
    dstband.FlushCache()
    dst_ds.FlushCache()

    # Clean up
    srcband = None
    dstband = None
    del src_ds, dst_ds


In [ ]:
def calculate_edge_tif_files(input_folder, output_folder):
    """
    Process .tif files by generating corresponding .tif filenames and calling compute_proximity.
    Parameters:
    input_folder (str): The path to the folder containing tif files.
    output_folder (str): The path to the folder where .tif files will be saved.
    """
    # List all raster files in the input folder
    raster_files = list_files_by_extension(input_folder, [".tiff", ".tif"])

    # Define the words to filter by
    filter_words = ["forest"]

    # Filter the raster files based on the presence of any of the filter words in their filenames
    filtered_raster_files = [
        file
        for file in raster_files
        if any(word in os.path.basename(file).lower() for word in filter_words)
    ]

    # Process each filtered raster file
    for raster_file in filtered_raster_files:
        # Extract the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(raster_file))[0]
        # Create the new .tif filename
        tif_filename = f"{base_name}_edge.tif"
        tif_path = os.path.join(output_folder, tif_filename)
        # Call compute_proximity with the original and new filenames
        # distance_to_no_forest_rasterio(raster_file, tif_path)
        distance_to_edge_gdal_no_mask(raster_file, tif_path)


In [ ]:
def calculate_distance_tif_files(input_folder, output_folder):
    """
    Process .tif files by generating corresponding .tif filenames and calling compute_proximity.
    Parameters:
    input_folder (str): The path to the folder containing tif files.
    output_folder (str): The path to the folder where .tif files will be saved.
    """
    # List all raster files in the input folder
    raster_files = list_files_by_extension(input_folder, [".tiff", ".tif"])

    # Define the words to filter by
    filter_words = ["rivers", "roads", "town"]

    # Filter the raster files based on the presence of any of the filter words in their filenames
    filtered_raster_files = [
        file
        for file in raster_files
        if any(word in os.path.basename(file).lower() for word in filter_words)
    ]

    # Process each filtered raster file
    for raster_file in filtered_raster_files:
        # Extract the base name of the file without extension
        base_name = os.path.splitext(os.path.basename(raster_file))[0]
        # Create the new .tif filename
        tif_filename = f"{base_name}_distance.tif"
        tif_path = os.path.join(output_folder, tif_filename)
        # Call compute_proximity with the original and new filenames
        # distance_from_outside(raster_file, tif_path)
        distance_to_edge_gdal_no_mask(raster_file, tif_path, 1)


In [ ]:
calculate_edge_tif_files(processed_data_folder, processed_data_folder)


In [ ]:
calculate_distance_tif_files(processed_data_folder, processed_data_folder)


## Calculate Forest Loss 

In [ ]:
import numpy as np
import rasterio


def process_forest_loss(input1_path, input2_path, output_path):
    # Open the input rasters
    with rasterio.open(input1_path) as src1:
        input1 = src1.read(1)
        bounds1 = src1.bounds
        profile = src1.profile
        nodata1 = src1.nodata

    with rasterio.open(input2_path) as src2:
        input2 = src2.read(1)
        bounds2 = src2.bounds
        nodata2 = src2.nodata

    # Check if the bounds of input1 are equal to or larger than those of input2
    if not (
        bounds1.left <= bounds2.left
        and bounds1.right >= bounds2.right
        and bounds1.top >= bounds2.top
        and bounds1.bottom <= bounds2.bottom
    ):
        raise ValueError(
            "The bounds of input1 must be equal to or larger than those of input2."
        )

    # Create masks for valid data
    valid_mask = (input1 != nodata1) & (input2 != nodata2)

    # Now only compare where both rasters have valid data
    output = np.full(input1.shape, 255, dtype=np.uint8)  # initialize with nodata (255)
    output[valid_mask] = ((input1 == 1) & (input2 == 0))[valid_mask].astype(np.uint8)

    # Update the profile for the output raster
    profile.update(dtype=rasterio.uint8, compress="deflate", nodata=255)

    # Write the output raster
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(output, 1)


In [ ]:
import numpy as np
import rasterio


def process_forest_loss(input1_path, input2_path, output_path):
    # Open the input rasters
    with rasterio.open(input1_path) as src1:
        input1 = src1.read(1)
        bounds1 = src1.bounds
        profile = src1.profile
        nodata1 = src1.nodata

    with rasterio.open(input2_path) as src2:
        input2 = src2.read(1)
        bounds2 = src2.bounds
        nodata2 = src2.nodata

    # Check if the bounds of input1 are equal to or larger than those of input2
    if not (
        bounds1.left <= bounds2.left
        and bounds1.right >= bounds2.right
        and bounds1.top >= bounds2.top
        and bounds1.bottom <= bounds2.bottom
    ):
        raise ValueError(
            "The bounds of input1 must be equal to or larger than those of input2."
        )

    # Create masks for valid data
    valid_mask = (input1 != nodata1) & (input2 != nodata2)

    # Initialize output with nodata (255)
    output = np.full(input1.shape, 255, dtype=np.uint8)

    # Set values based on conditions:
    # 1 where input1 == 1 and input2 == 0
    # 0 where input1 == 1 and input2 == 1
    # nodata (255) for all other cases

    # Create condition for 1s: input1 == 1 AND input2 == 0
    condition_1 = (input1 == 1) & (input2 == 0)

    # Create condition for 0s: input1 == 1 AND input2 == 1
    condition_0 = (input1 == 1) & (input2 == 1)

    # Apply conditions only where both inputs are valid
    output[valid_mask & condition_1] = 0
    output[valid_mask & condition_0] = 1

    # Update the profile for the output raster
    profile.update(dtype=rasterio.uint8, compress="deflate", nodata=255)

    # Write the output raster
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(output, 1)


In [ ]:
import re

# List all raster files in the input folder
forest_raster_files = list_files_by_extension(processed_data_folder, [".tiff", ".tif"])

# Define the words to filter by
filter_words = ["forest", "reprojected", forest_source]

# Define the words to exclude from the filtered files
exclude_words = ["distance", "edge"]

# Filter the raster files based on the presence of any of the filter words in their filenames
filtered_raster_files = [
    file
    for file in forest_raster_files
    if all(word in os.path.basename(file).lower() for word in filter_words)
    and not any(
        exclude_word in os.path.basename(file).lower() for exclude_word in exclude_words
    )
]


# Function to extract the year from a filename
def extract_year(filename):
    match = re.search(r"\d{4}", os.path.basename(filename))
    return int(match.group()) if match else None


# Sort the filtered raster files based on the extracted year in ascending order
sorted_raster_files = sorted(filtered_raster_files, key=extract_year)

sorted_raster_files  # Print the sorted list to verify


In [ ]:
def generate_output_filename(i1, i2):
    # Extract the base names from the input file paths
    base_name_i1 = i1.split("/")[-1].split(".")[0]
    base_name_i2 = i2.split("/")[-1].split(".")[0]

    # Find the common prefix up to the year
    def extract_common_prefix(base_name):
        common_prefix = ""
        for word in base_name.split("_"):
            if (
                word.isdigit() and len(word) == 4
            ):  # Check if the word is a four-digit year
                break
            common_prefix += word + "_"
        return common_prefix.strip("_")

    common_prefix = extract_common_prefix(base_name_i1)

    # Extract the years from the base names and ensure they are four digits
    def extract_year(base_name):
        year = next(
            (
                word
                for word in base_name.split("_")
                if word.isdigit() and len(word) == 4
            ),
            None,
        )
        if not year:
            raise ValueError(
                f"Year could not be extracted or is not four digits: {year}"
            )
        return year

    year_i1 = extract_year(base_name_i1)
    year_i2 = extract_year(base_name_i2)

    # Construct the output file name based on the input file names
    prefix = f"{common_prefix}_loss"
    suffix = f"{year_i1}_{year_i2}.tif"

    # Combine the directory path with the new file name
    dir_path = "/".join(i1.split("/")[:-1])
    output_filename = f"{dir_path}/{prefix}_{suffix}"

    return output_filename


In [ ]:
forest_loss1_filename = generate_output_filename(
    sorted_raster_files[0], sorted_raster_files[1]
)
process_forest_loss(
    sorted_raster_files[0], sorted_raster_files[1], forest_loss1_filename
)


In [ ]:
forest_loss2_filename = generate_output_filename(
    sorted_raster_files[0], sorted_raster_files[2]
)
process_forest_loss(
    sorted_raster_files[0], sorted_raster_files[2], forest_loss2_filename
)


In [ ]:
forest_loss3_filename = generate_output_filename(
    sorted_raster_files[1], sorted_raster_files[2]
)
process_forest_loss(
    sorted_raster_files[1], sorted_raster_files[2], forest_loss3_filename
)


In [ ]:
import numpy as np
import rasterio


def generate_deforestation_raster(
    raster1_path, raster2_path, raster3_path, output_path
):
    """
    Generate a deforestation raster from three input rasters.

    Parameters:
    - raster1_path: Path to the first raster file (period 1).
    - raster2_path: Path to the second raster file (period 2).
    - raster3_path: Path to the third raster file (period 3).
    - output_path: Path to save the output raster file.
    """

    # Open the input rasters
    with (
        rasterio.open(raster1_path) as src1,
        rasterio.open(raster2_path) as src2,
        rasterio.open(raster3_path) as src3,
    ):
        # Read the data into numpy arrays
        raster1 = src1.read(1)
        raster2 = src2.read(1)
        raster3 = src3.read(1)

        # Create an output array initialized with NoData value (0)
        output_raster = np.zeros_like(raster1, dtype=np.uint8)

        # Set the values based on deforestation periods
        output_raster[(raster1 == 1) & (raster2 == 0)] = (
            1  # Deforestation in period 1-2
        )
        output_raster[(raster2 == 1) & (raster3 == 0)] = (
            2  # Deforestation in period 2-3
        )
        # Set the remaining forest value only where no deforestation has been marked
        output_raster[(output_raster == 0) & (raster3 == 1)] = (
            3  # Remaining forest in period 3
        )

    # Define the metadata for the output raster
    meta = src1.meta
    meta.update({"count": 1, "dtype": np.uint8, "nodata": 0, "compress": "deflate"})

    # Write the output raster to a file
    with rasterio.open(output_path, "w", **meta) as dst:
        dst.write(output_raster, 1)


In [ ]:
def generate_output_filename2(i1, i2, i3):
    # Extract the base names from the input file paths
    base_name_i1 = i1.split("/")[-1].split(".")[0]
    base_name_i2 = i2.split("/")[-1].split(".")[0]
    base_name_i3 = i3.split("/")[-1].split(".")[0]

    # Find the common prefix up to the year
    def extract_common_prefix(base_name):
        common_prefix = ""
        for word in base_name.split("_"):
            if (
                word.isdigit() and len(word) == 4
            ):  # Check if the word is a four-digit year
                break
            common_prefix += word + "_"
        return common_prefix.strip("_")

    common_prefix = extract_common_prefix(base_name_i1)

    # Extract the years from the base names and ensure they are four digits
    def extract_year(base_name):
        year = next(
            (
                word
                for word in base_name.split("_")
                if word.isdigit() and len(word) == 4
            ),
            None,
        )
        if not year:
            raise ValueError(
                f"Year could not be extracted or is not four digits: {year}"
            )
        return year

    year_i1 = extract_year(base_name_i1)
    year_i2 = extract_year(base_name_i2)
    year_i3 = extract_year(base_name_i3)

    # Construct the output file name based on the input file names
    prefix = f"{common_prefix}_loss"
    suffix = f"{year_i1}_{year_i2}_{year_i3}.tif"

    # Combine the directory path with the new file name
    dir_path = "/".join(i1.split("/")[:-1])
    output_filename = f"{dir_path}/{prefix}_{suffix}"

    return output_filename


In [ ]:
total_forest_loss_filename = generate_output_filename2(
    sorted_raster_files[0], sorted_raster_files[1], sorted_raster_files[2]
)
total_forest_loss = generate_deforestation_raster(
    sorted_raster_files[0],
    sorted_raster_files[1],
    sorted_raster_files[2],
    total_forest_loss_filename,
)
